In [1]:
import numpy as np
import gurobipy as gp

In [2]:
M = 10000#np.sum([0,132,217,164,58,132,0,290,201,79,217,290,0,113,303,164,201,113,0,196,58,79,303,196,0])
N = 5
# variables are x11, x12, x13, ..., x1N, x21, x22, x23, ..., x2N, ..., xN1, xN2, ...., xNN, u1, u2, ..., uN
obj = np.array([M,132,217,164,58,132,M,290,201,79,217,290,M,113,303,164,201,113,M,196,58,79,303,196,M]+[0]*N)

In [3]:
A = np.zeros((2*N + (N-1)**2 - (N-1),N**2 + N))
b = np.zeros(2*N + (N-1)**2 - (N-1))
direction = np.array(['']*(2*N + (N-1)**2 - (N-1)))

In [5]:
ind_vec = np.array(range(N))
ind_vec

array([0, 1, 2, 3, 4])

In [6]:
j = 3
j*N + ind_vec
x15 + x16 + x17 + x18 + x19 = 1

array([15, 16, 17, 18, 19])

In [ ]:
x00 x01 x02 ... x0(N-1)
x10 x11 x12 ... x2(n-1)
...
xn1 xn2 xn3 ... xnn

In [ ]:
x11 x12 x13 ... x1N x21 x22 x23 ... x2n ... xn1 xn2 xn3 ... xnn

In [ ]:
x12 is index N+2

xij

i*N + j


In [7]:
i = 3
ind_vec*N + i

array([ 3,  8, 13, 18, 23])

In [4]:
ind_vec = np.array(range(N))
# leave each city once
row = 0
for j in range(N):
    A[row,j*N + ind_vec] = 1
    b[row] = 1
    direction[row] = '='
    row+=1

In [5]:
# enter each city once
for i in range(N):
    A[row,ind_vec*N + i] = 1
    b[row] = 1
    direction[row] = '='
    row+=1

In [6]:
for i in range(1,N):
    for j in range(1,N):
        if i != j:
            # ui - uj + Nxij <= (N-1)
            A[row,[N**2+i,N**2+j,j*N+i]] = [1,-1,N]
            b[row] = N-1
            direction[row] = '<'
            row+=1

In [7]:
tspMod = gp.Model()
tspMod_x = tspMod.addMVar(N**2 + N,vtype=['B']*(N**2)+['C']*N) 
tspMod_con = tspMod.addMConstrs(A, tspMod_x, direction, b)
tspMod.setMObjective(None,obj,0,sense=gp.GRB.MINIMIZE)

tspMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
tspMod.optimize()

Academic license - for non-commercial use only - expires 2022-08-22
Using license file /Users/dan/gurobi.lic


In [8]:
tspMod_x.x

array([-0., -0., -0., -0.,  1., -0., -0., -0.,  1., -0.,  1., -0., -0.,
       -0., -0., -0., -0.,  1., -0., -0., -0.,  1., -0., -0., -0.,  0.,
        2.,  0.,  1.,  3.])

In [9]:
tspMod.objval

668.0

In [10]:
where = 0
miles_traveled = 0
print('Start in city 0.')
for city in range(N):
    where_new = np.where(tspMod_x.x[(where*N):((where+1)*N)])[0][0]
    dist = obj[where*N+where_new]
    miles_traveled += dist
    print('Then go to city '+str(where_new)+', traveling '+str(dist)+' miles along the way.')
    where = where_new
print('For a total of '+str(miles_traveled)+' miles.')

Start in city 0.
Then go to city 4, traveling 58 miles along the way.
Then go to city 1, traveling 79 miles along the way.
Then go to city 3, traveling 201 miles along the way.
Then go to city 2, traveling 113 miles along the way.
Then go to city 0, traveling 217 miles along the way.
For a total of 668 miles.


In [ ]:
# this path may not be unique, there are other paths that are optimal
# but they'll all have the same total miles traveled!
# for example 0 -> 2 -> 3 -> 1 -> 4 -> 0
# also takes 668 miles!

In [ ]:
seq = [0, 2, 3, 1, 4, 0]
miles_traveled2 = 0
for loc in range(N):
    where = seq[loc]
    where_new = seq[loc+1]
    dist = obj[where*N+where_new]
    miles_traveled2 += dist
print(miles_traveled2)

In [ ]:
# I have no idea how many other paths are optimal...